In [1]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import datetime
import json
import psycopg2

# load the credentials from the JSON file
with open('config/credentials.json') as f:
    credentials = json.load(f)

connection_string = f"postgresql://{credentials['username']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database_name']}"

# create the engine with the connection string
engine = create_engine(connection_string)


C:\Users\domin\anaconda3\envs\car_sharing_env\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
### pre-processing of reservation table: ###
# reservationfrom_discrete: discretize reservationfrom to 15 minutes 
# reservationto_discrete: discretize reservationto to 15 minutes 
# syscreatedate_discrete: discretize syscreatedate to 15 minutes 
# time_booking_to_bookingstart: time difference between syscreatedate and reservationfrom, in 15 min units
# reservation_duration: time difference between reservationfrom_discrete and reservationto_discrete, in 15 min units
# syscreatedate_daytime: time of the day of syscreatedate, in 15 min units
# reservationfrom_daytime: time of the day of reservationfrom, in 15 min units
# reservationto_daytime: time of the day of reservationto, in 15 min units
# booking_trip_same_day: boolean wether a trip was reserved on the same day as the trip or not

sql = """WITH stations AS (
SELECT *,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(syscreatedate, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as syscreatedate_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationfrom, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationfrom_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationto, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationto_discrete
FROM mobility.reservation 
)
SELECT *, 
EXTRACT(epoch FROM (reservationfrom_discrete - syscreatedate_discrete)) / 900 AS time_booking_to_bookingstart,
EXTRACT(epoch FROM (reservationto_discrete - reservationfrom_discrete)) / 900 AS reservation_duration
FROM stations
ORDER BY reservationfrom   
"""

data = pd.read_sql(sql, engine)
data["syscreatedate_daytime"] = data.syscreatedate_discrete.apply(lambda x: x.hour) * 4 + data.syscreatedate_discrete.apply(lambda x: x.minute) / 15
data["reservationfrom_daytime"] = data.reservationfrom_discrete.apply(lambda x: x.hour) * 4 + data.reservationfrom_discrete.apply(lambda x: x.minute) / 15
data["reservationto_daytime"] = data.reservationto_discrete.apply(lambda x: x.hour) * 4 + data.reservationto_discrete.apply(lambda x: x.minute) / 15
data["syscreatedate_discrete_date"] = pd.to_datetime(data['syscreatedate_discrete']).dt.date 
data["reservationfrom_discrete_date"] = pd.to_datetime(data['reservationfrom_discrete']).dt.date 
data["booking_trip_same_day"] = data["reservationfrom_discrete_date"] == data["syscreatedate_discrete_date"] 
data["syscreatedate_daytime_endofday"] = data["syscreatedate_daytime"]
data["reservationfrom_daytime_endofday"] = data["reservationfrom_daytime"]
data["reservationto_daytime_endofday"]  = data["reservationto_daytime"] 

cond = data["reservationfrom_discrete_date"] > data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_daytime"] = 0

cond = data["reservationfrom_discrete_date"] <= data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_daytime_endofday"] = 96

cond = data["reservationfrom_discrete_date"] > data["syscreatedate_discrete_date"] + pd.Timedelta(days=1)
data.loc[cond, "syscreatedate_daytime_endofday"] = 0

cond = data["reservationfrom_daytime_endofday"] == 0
data.loc[cond, "reservationfrom_daytime_endofday"] = 96


cond2 = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond2, "syscreatedate_daytime_endofday"] = data["reservationfrom_daytime_endofday"]


cond2 = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond2, "syscreatedate_daytime"] = data["reservationfrom_daytime"]

data.to_sql("reservations_discrete", engine, schema="msc_2023_dominik", if_exists='replace')
data.head()

In [40]:
### create indices for fast data loading in environment ###
conn = psycopg2.connect(dbname=credentials['database_name'], user=credentials['username'], password=credentials['password'], host=credentials['host'])
cur = conn.cursor()
#cur.execute("CREATE INDEX reservationfrom_discrete_date ON msc_2023_dominik.reservations_discrete USING hash(reservationfrom_discrete_date)")
#cur.execute("CREATE INDEX reservationfrom_discrete_index_b_tree ON msc_2023_dominik.reservations_discrete (reservationfrom_discrete)")
#for i in range(0,83):
#    cur.execute("CREATE INDEX vehicle_no_b_tree_index_week_{} ON msc_2023_dominik.discrete_weeks_{} (vehicle_no)".format(i,i))
    
conn.commit()
cur.close()
conn.close()

In [8]:
### pre-processing of station tables: ###
# Contains all stations in the mobility.station table with valid geometry, that were used during the 
# observation period for bookings (mobility.reservation and mobility.service_reservation)

sql = "WITH stations AS (SELECT DISTINCT start_station_no FROM (SELECT DISTINCT start_station_no FROM mobility.reservation UNION SELECT DISTINCT start_station_no FROM mobility.service_reservation) AS station_union) SELECT station_no, ST_Transform(ST_SetSRID(geom,4326), 2056) AS geom FROM stations LEFT OUTER JOIN mobility.station ON start_station_no = station_no WHERE geom is not NULL"
gdf = gpd.read_postgis(sql, engine, geom_col='geom',crs = "EPSG:2056")
gdf.to_postgis("distinct_stations", engine, schema="msc_2023_dominik", if_exists='replace')  

In [18]:
### pre-processing of vehicle tables: ###
# get information about each car used in the discrete tables
# replace some car types, which are not used anymore (Budget Electro by Budget, Combi Electro by Combi)
sql = """SELECT vehicle_no, mobility.simulated_ev.model_name, mobility.simulated_ev.brand_name, charge_power, battery_capacity, range, vehicle_category 
         FROM mobility.simulated_ev 
         FULL OUTER JOIN mobility.vehicle USING (vehicle_no) 
         RIGHT OUTER JOIN discrete_mobility.discrete_weeks_0 using (vehicle_no)"""
data = pd.read_sql(sql, engine)
data = data.replace(["Budget Electro"], 'Budget')
data = data.replace(['Combi Electro'], 'Combi')
data.to_sql("vehicles_preprocessing", engine, schema="msc_2023_dominik", if_exists='replace')

### vehicle types: ###
# get car information for each category
sql = """WITH vehicles AS (
                            SELECT *
                            FROM msc_2023_dominik.vehicles_preprocessing 
        )
        SELECT model_name, brand_name, charge_power, battery_capacity, range, vehicle_category 
        FROM vehicles
        WHERE model_name IS NOT NULL
        GROUP by model_name, brand_name, charge_power, battery_capacity, range, vehicle_category 
        ORDER BY vehicle_category"""
data = pd.read_sql(sql, engine)
data.to_sql("vehicles_types", engine, schema="msc_2023_dominik", if_exists='replace')

### Vehicle Information table: ###
# create final vehicle information table; containing all vehicle informatino for each car in the discrete tables
sql = """
WITH vehicles AS (
    SELECT vehicle_no, vehicle_category
    FROM msc_2023_dominik.vehicles_preprocessing 
), 
vehicle_categories AS (
    SELECT *
    FROM msc_2023_dominik.vehicles_types
)
SELECT *
FROM vehicles JOIN vehicle_categories USING(vehicle_category)
"""
data = pd.read_sql(sql, engine)
data = data.drop(["index"], axis = 1)
data.to_sql("vehicle_information", engine, schema="msc_2023_dominik", if_exists='replace')

233